In [1]:
from astropy.io import fits
import numpy as np

In [2]:
%pwd

'/scratch/wyz5rge/synthetic-hr/2024/08-17/merged_model/generate_model'

In [3]:
import merge_files

In [4]:
target_filepath = 'iso_6.00.fits'
# baraffe_filepath = 'data/baraffe/z015/iso_6.00.fits'
baraffe_filepath = 'data/baraffe/z015/iso_6.00.dat'
merged_filepath = 'data/merged/z015_norot/iso_6.00.fits'

merge_files.main(target_filepath, baraffe_filepath, merged_filepath)

In [5]:
hdul = fits.open('iso_6.00.fits')
hdul.info()

print(hdul[1].columns)
print(hdul[1].data)

Filename: iso_6.00.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     24   1178R x 8C   [D, D, D, D, D, D, K, 12A]   
ColDefs(
    name = 'col1'; format = 'D'
    name = 'col2'; format = 'D'
    name = 'col3'; format = 'D'
    name = 'col4'; format = 'D'
    name = 'col5'; format = 'D'
    name = 'col6'; format = 'D'
    name = 'col7'; format = 'K'
    name = 'col8'; format = '12A'
)
[(1.00000e-02, 3.3704, -2.6966, 3.5697, 3.3704, 0.01 , 1, 'Baraffe')
 (1.50000e-02, 3.3986, -2.4196, 3.5815, 3.3986, 0.015, 1, 'Baraffe')
 (2.00000e-02, 3.4147, -2.2427, 3.594 , 3.4147, 0.02 , 1, 'Baraffe') ...
 (3.86487e+02, 4.6325,  6.9265, 3.579 , 4.6325, 0.014, 1, 'Ekstrom')
 (4.39595e+02, 4.6158,  6.9902, 3.5042, 4.6158, 0.014, 1, 'Ekstrom')
 (5.00000e+02, 4.5939,  7.0524, 3.4104, 4.5939, 0.014, 1, 'Ekstrom')]


In [6]:
# import packages
import numpy as np
import pylab as py
import pdb
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# import nbody6tools reader to read data
from nbody6tools import Reader

# import nbody62spisea to retrieve star ages
from nbody62spisea import converter

/home/wyz5rge/.local/lib/python3.6/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


In [11]:
# define isochrone parameters
AKs = 0
dist = 410
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F115W', 'jwst,F182M']
filters = ['m_jwst_F115W', 'm_jwst_F182M']
iso_dir = 'isochrones/'
metallicity = 0

model_iso = []

baraffe = evolution.Baraffe15()
model_iso.append(baraffe)

merged_model = evolution.MergedBaraffePisaEkstromParsec()
model_iso.append(merged_model)

new_merged_model = evolution.MergedLowMassBaraffePisaEkstromParsec()
model_iso.append(new_merged_model)

In [12]:
iso_arr = []

age = 6.00

for i in range(len(model_iso)):
    iso = synthetic.IsochronePhot(age, AKs, dist, metallicity=metallicity,
                            evo_model=model_iso[i], atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    iso_arr.append(iso)

Isochrone generation took 1.111270 s.
Making photometry for isochrone: log(t) = 6.00  AKs = 0.00  dist = 410
     Starting at:  2024-08-25 22:18:49.405113   Usually takes ~5 minutes
Starting filter: jwst,F115W   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F115W = 17.72
Starting filter: jwst,F182M   Elapsed time: 0.62 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F182M = 16.97
      Time taken: 1.24 seconds
Changing to logg=4.50 for T= 39129 logg=4.17
Changing to logg=4.50 for T= 39301 logg=4.17
Changing to logg=4.50 for T= 39464 logg=4.17
Changing to logg=4.50 for T= 39637 logg=4.16
Changing to logg=4.50 for T= 39802 logg=4.16
Changing to logg=4.50 for T= 39976 logg=4.16
Changing to logg=4.50 for T= 40142 logg=4.16
Changing to logg=4.50 for T= 40300 logg=4.16
Changing to logg=4.50 for T= 40309 logg=4.16
Changing to logg=4.50 for T= 40448 logg=4.15
Changing to logg=4.50 for T= 40588 logg=4.15
Changing to lo

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/wyz5rge/models//evolution/merged/low_mass_baraffe_pisa_ekstrom_parsec/z015_rot/iso_6.01.fits'

In [ ]:
# Plot CMD
fig, axes = py.subplots(figsize=(15, 10))
py.subplot(1, 2, 1)

colors = ['r', 'b', 'orange']
labels = ['Baraffe', 'Merged', 'Merged (Low mass)']

for i in range(len(iso_arr)):
    if i == 4:
        lw = 4
        zorder = -10
        alpha = 0.5
    else:
        lw = 1
        zorder = None
        alpha = 1
    py.plot(iso_arr[i].points[filters[0]] - iso_arr[i].points[filters[1]], 
           iso_arr[i].points[filters[1]], color=colors[i], label=labels[i], lw = lw, zorder = zorder, alpha = alpha)

idx = np.where(abs(iso_arr[1].points['mass'] - 0.5) == min(abs(iso_arr[1].points['mass'] - 0.5)))[0].item()
py.plot(iso_arr[1].points[filters[0]][idx] - iso_arr[1].points[filters[1]][idx], 
       iso_arr[1].points[filters[1]][idx], 'b*', ms=15, label='0.5 $M_\odot$')

idx = np.where(abs(iso_arr[1].points['mass'] - 6) == min(abs(iso_arr[1].points['mass'] - 6)))[0].item()
py.plot(iso_arr[1].points[filters[0]][idx] - iso_arr[1].points[filters[1]][idx], 
       iso_arr[1].points[filters[1]][idx], 'r*', ms=15, label='6 $M_\odot$')

py.gca().invert_yaxis()
py.legend()